[View in Colaboratory](https://colab.research.google.com/github/hamil168/Chatbots/blob/master/Preproc.ipynb)

In [0]:
# For a fresh Colab instance, clone fresh:
!pip install -q xlrd
!git clone https://github.com/hamil168/Chatbots

Cloning into 'Chatbots'...
remote: Counting objects: 23, done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 23 (delta 8), reused 13 (delta 2), pack-reused 0
Unpacking objects: 100% (23/23), done.


In [21]:
# Change to Colab directory:
!cd Chatbots/

# For an existing Colab instance, pull from master
#!git pull https://github.com/hamil168/Chatbots master

/bin/sh: 1: cd: can't cd to Chatbots/


In [25]:
# Files as they appear in the repo clone
ls

Cornell Movie Script Database EDA.ipynb  movie_lines.txt
movie_conversations.txt                  Preproc.ipynb


In [26]:
# Start script

import numpy as np
import tensorflow as tf
import re
import time

In [27]:
lines = open('movie_lines.txt', encoding = 'utf-8', errors = 'ignore').read().split('\n')
conversations = open('movie_conversations.txt', encoding = 'utf-8', errors = 'ignore').read().split('\n')

In [32]:
print("\n".join(lines[0:10]))

L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!
L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!
L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.
L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?
L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go.
L924 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ Wow
L872 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Okay -- you're gonna need to learn how to lie.
L871 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ No
L870 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I'm kidding.  You know how sometimes you just become this "persona"?  And you don't know how to quit?
L869 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Like my fear of wearing pastels?


In [33]:
print("\n".join(conversations[0:10]))

u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L194', 'L195', 'L196', 'L197']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L198', 'L199']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L200', 'L201', 'L202', 'L203']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L204', 'L205', 'L206']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L207', 'L208']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L271', 'L272', 'L273', 'L274', 'L275']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L276', 'L277']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L280', 'L281']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L363', 'L364']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L365', 'L366']


In [0]:
**TODO**: Create Dictionary to map lines to conversations
  
Goal is a dataset that contains 2 columns, input for NN, output is target for supervised learning
map each line with its ID



In [34]:
# iterate through each line, split into different elements, get key, get value
id2line = {}

for line in lines:
  _line = line.split(' +++$+++ ')
  
  if len(_line) == 5:
    id2line[_line[0]] = _line[4]
  

In [44]:
id2line['L1045']

'They do not!'

In [45]:
# Clean up conversations file: 

print("\n".join(conversations[0:2]))

u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L194', 'L195', 'L196', 'L197']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L198', 'L199']


In [46]:
# create a list of the conversations
conversations_ids = []

# The last row of data set is empty, so skip it
for conversation in conversations[:-1]:
  
  # Split and remove brackets
  # Remove single quote
  # Remove spaces
  _conversation = conversation.split(' +++$+++ ')[-1][1:-1].replace("'","").replace(" ","")
  
  # Append as list by spliting on commas
  conversations_ids.append(_conversation.split(","))

In [47]:
conversations_ids[0:10]

[['L194', 'L195', 'L196', 'L197'],
 ['L198', 'L199'],
 ['L200', 'L201', 'L202', 'L203'],
 ['L204', 'L205', 'L206'],
 ['L207', 'L208'],
 ['L271', 'L272', 'L273', 'L274', 'L275'],
 ['L276', 'L277'],
 ['L280', 'L281'],
 ['L363', 'L364'],
 ['L365', 'L366']]

In [0]:
# Make lists for "questions" and "answers"
# A "question" is any line in a conversation list, "answer" is the line that follows
# Note, their numbers are always sequential

In [48]:
questions = []
answers = []

for conversation in conversations_ids:
  
  for i in range(len(conversation) - 1):
    questions.append(id2line[conversation[i]])
    answers.append(id2line[conversation[i+1]])

In [62]:
questions[10]

"C'esc ma tete. This is my head"

In [63]:
answers[10]

"Right.  See?  You're ready for the quiz."

In [141]:
# now we can start cleaning with text
# Look up ways of dealing with contractions
# This is just a rough start

def clean_text(text):
  text = text.lower()
  text = re.sub(r"i'm", "i am", text)
  text = re.sub(r"he's", "he is", text)
  text = re.sub(r"she's", "she is", text)
  text = re.sub(r"that's", "that is", text)
  text = re.sub(r"what's", "what is", text)
  text = re.sub(r"where's", "where is", text)
  
  text = re.sub(r"\'ll", " will", text)
  text = re.sub(r"\'ve", " have", text)
  text = re.sub(r"\'d", " would", text)
  text = re.sub(r"won't", "will not", text)
  text = re.sub(r"can't", "can not", text)
  text = re.sub(r"don't", "do not", text)
  
  text = re.sub(r"\'re", " are", text)
  
  text = re.sub(r"[-()\'#/@;:<>{}'\+\=\-\|.?,\!]", "", text)
  
  #text = re.sub(r"[ ]*", " ", text)
 

  return text

In [142]:
questions[0:2]

['Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.',
 "Well, I thought we'd start with pronunciation, if that's okay with you."]

In [143]:
[clean_text(question) for question in questions[0:2]]

['can we make this quick  roxanne korrine and andrew barrett are having an incredibly horrendous public break up on the quad  again',
 'well i thought we would start with pronunciation if that is okay with you']

In [144]:
# Apply on all "questions" and "answers"

clean_questions = [clean_text(question) for question in questions]
clean_answers = [clean_text(answer) for answer in answers]

In [145]:
print(questions[10])
print(clean_questions[10])

C'esc ma tete. This is my head
cesc ma tete this is my head


In [146]:
print(answers[10])
print(clean_answers[10])

Right.  See?  You're ready for the quiz.
right  see  you are ready for the quiz


In [146]:
# BoW

# Examine words that appear < 5% of the corpous
# create dictionary to map each word to number of occurrences

In [149]:
word2count = {}
for question in clean_questions:
  for word in question.split():
    if word not in word2count:
      word2count[word] = 1
    else:
      word2count[word] += 1
      
# This double counts some entries... this may be OK. come back to think on it later.
for answer in clean_answers:
  for word in answer.split():
    if word not in word2count:
      word2count[word] = 1
    else:
      word2count[word] += 1

In [151]:
word2count['thought']

4560

In [152]:
# Create 2 dictionaries that map questions words and answer words to integers


threshold = 20

questionswords2int = {}
word_number = 0

for word, count in word2count.items():
    if count >= threshold:
      questionswords2int[word] = word_number
      word_number += 1


answerswords2int = {}
word_number = 0

for word, count in word2count.items():
    if count >= threshold:
      answerswords2int[word] = word_number
      word_number += 1    

In [159]:
questionswords2int.items()

dict_items([('can', 0), ('we', 1), ('make', 2), ('this', 3), ('quick', 4), ('and', 5), ('andrew', 6), ('barrett', 7), ('are', 8), ('having', 9), ('an', 10), ('incredibly', 11), ('public', 12), ('break', 13), ('up', 14), ('on', 15), ('the', 16), ('again', 17), ('well', 18), ('i', 19), ('thought', 20), ('would', 21), ('start', 22), ('with', 23), ('if', 24), ('that', 25), ('is', 26), ('okay', 27), ('you', 28), ('not', 29), ('part', 30), ('please', 31), ('asking', 32), ('me', 33), ('out', 34), ('so', 35), ('cute', 36), ('what', 37), ('your', 38), ('name', 39), ('no', 40), ('its', 41), ('my', 42), ('fault', 43), ('didnt', 44), ('have', 45), ('a', 46), ('proper', 47), ('cameron', 48), ('thing', 49), ('am', 50), ('at', 51), ('mercy', 52), ('of', 53), ('particularly', 54), ('hideous', 55), ('breed', 56), ('loser', 57), ('sister', 58), ('date', 59), ('until', 60), ('she', 61), ('does', 62), ('why', 63), ('mystery', 64), ('used', 65), ('to', 66), ('be', 67), ('really', 68), ('popular', 69), ('wh

In [160]:
answerswords2int.items()

dict_items([('can', 0), ('we', 1), ('make', 2), ('this', 3), ('quick', 4), ('and', 5), ('andrew', 6), ('barrett', 7), ('are', 8), ('having', 9), ('an', 10), ('incredibly', 11), ('public', 12), ('break', 13), ('up', 14), ('on', 15), ('the', 16), ('again', 17), ('well', 18), ('i', 19), ('thought', 20), ('would', 21), ('start', 22), ('with', 23), ('if', 24), ('that', 25), ('is', 26), ('okay', 27), ('you', 28), ('not', 29), ('part', 30), ('please', 31), ('asking', 32), ('me', 33), ('out', 34), ('so', 35), ('cute', 36), ('what', 37), ('your', 38), ('name', 39), ('no', 40), ('its', 41), ('my', 42), ('fault', 43), ('didnt', 44), ('have', 45), ('a', 46), ('proper', 47), ('cameron', 48), ('thing', 49), ('am', 50), ('at', 51), ('mercy', 52), ('of', 53), ('particularly', 54), ('hideous', 55), ('breed', 56), ('loser', 57), ('sister', 58), ('date', 59), ('until', 60), ('she', 61), ('does', 62), ('why', 63), ('mystery', 64), ('used', 65), ('to', 66), ('be', 67), ('really', 68), ('popular', 69), ('wh